# Final Project

In [146]:
from datetime import date
today = date.today()
d2 = today.strftime("%B %d, %Y")
print("Updated by Alfonso Torres-Rua, ", d2)

Updated by Alfonso Torres-Rua,  April 27, 2020


# Spacial Resolution

In [147]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

# importing ipygee for dynamic mapping
from ipygee import *

Map = Map() # from ipygee
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

# Modis

In [148]:
myd09 = ee.ImageCollection("MODIS/006/MYD09GA")

# Define a location of interest as a point at SFO airport.
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="a.torres@aggiemail.usu.edu") #put your email here
# address, (latitude, longitude) = geolocator.geocode("Logan Utah")
# print(latitude, longitude)


latitude = 41.7370
longitude = -111.8338
sfoPoint = ee.Geometry.Point([-111.8338, 41.7370]);

#Define a region to display (from the entire planet to a small region). USE THE COORDINATES FROM GOOGLE EARTH PRO
# area_to_display = ee.Geometry.Rectangle([-179, 80, 179, -80]) #entire planet

In [149]:
# Get a surface reflectance image from the MODIS MYD09GA collection.
modisImage = ee.Image(myd09.filterDate('2019-07-06').first());

modisImage = modisImage.multiply(0.0001)
# Print the image band names to the console.
band_names = modisImage.bandNames()
print(band_names.getInfo())

['num_observations_1km', 'state_1km', 'SensorZenith', 'SensorAzimuth', 'Range', 'SolarZenith', 'SolarAzimuth', 'gflags', 'orbit_pnt', 'granule_pnt', 'num_observations_500m', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'QC_500m', 'obscov_500m', 'iobs_res', 'q_scan']


In [150]:
# Use these MODIS bands for red, green, blue, respectively.
modisBands = ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'];

#// Add the image to the map.
# url = modisImage.getThumbURL({'dimensions': 700,'bands':modisBands,'min': 0, 'max': 3000,'region':area_to_display})
# # print(url)
# Image(url=url)

# add layer
Map.addLayer(modisImage, {'bands':modisBands,'min': 0, 'max': 1}, name='MODIS Image')

In [151]:
# Get the scale of the data from the first band's projection:
modisScale = modisImage.select('sur_refl_b01').projection().nominalScale();

print('MODIS scale:', modisScale.getInfo());


MODIS scale: 463.3127165275


In [152]:
# add layer SensorZenith
Map.addLayer(modisImage, {'bands':'SensorZenith','min': 0, 'max': 0.3}, name='MODIS SensorZenith')

# Landsat

In [153]:
# Filter MSS imagery by location, date and cloudiness.
mss= ee.ImageCollection("LANDSAT/LM05/C01/T2")

mssImage = ee.Image(mss
    .filterBounds(sfoPoint)
    .sort('CLOUD_COVER')
#Get the least cloudy image.
    .first());

# Print the image band names to the console.
band_names = mssImage.bandNames()
print(band_names.getInfo())
# Display the MSS image as a color-IR composite.


['B1', 'B2', 'B3', 'B4', 'BQA']


In [154]:
# Use these MODIS bands for red, green, blue, respectively.
mssBands = ['B3', 'B2', 'B1'];

#// Add the image to the map.
# url = mssImage.getThumbURL({'dimensions': 700,'bands':mssBands,'min': 0, 'max': 200}) # because the area is small we don't need region
# # print(url)
# Image(url=url)

Map.addLayer(mssImage, {'bands':mssBands,'min': 0, 'max': 200}, name='LandsatMSS Image')

bounds = sfoPoint.buffer(70000)
Map.centerObject(bounds)

In [155]:
# landsat_area = ee.Geometry.Rectangle([-123.4, 38.5,-120.7, 36.5]) #Define upper left, lower right corner of Landsat image over san Francisco.
# #// Add the image to the map.
# modisBands = ['sur_refl_b02', 'sur_refl_b01', 'sur_refl_b04'];

# url = modisImage.getThumbURL({'dimensions': 700,'bands':modisBands,'min': 0, 'max': 6000,'region':landsat_area})
# # print(url)
# Image(url=url)

In [156]:
# Get the scale of the MSS data from its projection:
mssScale = mssImage.select('B1').projection().nominalScale();

print('MSS scale:', mssScale.getInfo());


MSS scale: 60


# TM

In [157]:
tm = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")

# // Filter TM imagery by location, date and cloudiness.
tmImage = ee.Image(tm
    .filterBounds(sfoPoint) # I could have used sfopoint too. every image has a boundary geometry
    .filterDate('2011-05-01', '2011-10-01')
    .sort('CLOUD_COVER') # not all satellites have this property, check the properties table
    .first());

# Print the image band names to the console.
band_names = tmImage.bandNames()
print(band_names.getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA']


In [158]:
# // Display the TM image as a color-IR composite.
tmBands=  ['B4', 'B3', 'B2']
#// Add the image to the map.
# url = tmImage.getThumbURL({'dimensions': 700,'bands':tmBands,'min': 0, 'max': 0.4}) # because the area is small we don't need region
# # print(url)
# Image(url=url)

Map.addLayer(tmImage, {'bands':tmBands,'min': 0, 'max': 0.4}, name='Landsat TM Image')

bounds = sfoPoint.buffer(70000)
Map.centerObject(bounds)


Check the scale (in meters) as previously:

In [159]:
# // Get the scale of the TM data from its projection:
tmScale = tmImage.select('B1').projection().nominalScale();
print('TM scale:', tmScale.getInfo());


TM scale: 30


In [160]:
## - Sentinel 2 

# NAIP

In [161]:
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# // Get NAIP images for the study period and region of interest.
naipImages = naip.filterDate('2012-01-01', '2019-12-31').filterBounds(sfoPoint);

# // Mosaic adjacent images into a single image.
naipImage = naipImages.mosaic();

# Print the image band names to the console.
band_names = naipImage.bandNames()
print(band_names.getInfo())


['R', 'G', 'B', 'N']


In [162]:
#Define a region to display (from the entire planet to a small region). USE THE COORDINATES FROM GOOGLE EARTH PRO
# area_to_display = ee.Geometry.Rectangle([121.5, 41.0, 122.5, 42.2])

# // Display the NAIP mosaic as a color-IR composite.
nBands=  ['N', 'R', 'G']
#// Add the image to the map.
# url = naipImage.getThumbURL({'dimensions': 700,'bands':nBands,'min': 0, 'max': 255, 'region':area_to_display}) # because the area is small we don't need region
# # print(url)
# Image(url=url)

Map.addLayer(naipImage, {'bands':nBands,'min': 0, 'max': 255}, name='NAIP Image')

bounds = sfoPoint.buffer(70000)
Map.centerObject(bounds)

In [163]:
# // Get the NAIP resolution from the first image in the mosaic.
naipScale = ee.Image(naipImages.first()).projection().nominalScale();
print('NAIP scale:', naipScale.getInfo());


NAIP scale: 0.6


# ASTER

In [164]:
aster = ee.ImageCollection('ASTER/AST_L1T_003')

asterImages = aster.filterBounds(sfoPoint).filterDate('2019-01-01', '2019-12-31')

asterImage = asterImages.mosaic();
band_names = asterImage.bandNames()
print(band_names.getInfo())

asterBands = ['B01', 'B02', 'B3N']

Map.addLayer(asterImage, {'bands':asterBands,'min': 0.0, 'max': 255.0}, name='ASTER Image')



['B01', 'B02', 'B3N', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14']


# Sentinal-2

In [165]:
s2 = ee.ImageCollection('COPERNICUS/S2')

s2Image = ee.Image(s2.filterBounds(sfoPoint).sort('CLOUDY_PIXEL_PERCENTAGE').first())
s2Image = s2Image.multiply(0.0001)


band_names = s2Image.bandNames()
print(band_names.getInfo())

s2Bands = ['B5', 'B6', 'B7']

Map.addLayer(s2Image, {'bands':s2Bands, 'min': 0.0, 'max': 0.3}, name = 'Sentinal-2 Image')

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


# Sentinal-3

In [166]:
s3 = ee.ImageCollection("COPERNICUS/S3/OLCI") 

s3Image = ee.Image(s3.filterDate('2019-07-01', '2019-07-02').filterBounds(sfoPoint).first())
band_names = s3Image.bandNames()
print(band_names.getInfo())

s3Bands = ['Oa01_radiance', 'Oa02_radiance', 'Oa03_radiance']

Map.addLayer(s3Image, {'bands':s3Bands, 'min': 0.0, 'max': 255.0}, name = 'Sentinal-3 Image')

['Oa01_radiance', 'Oa02_radiance', 'Oa03_radiance', 'Oa04_radiance', 'Oa05_radiance', 'Oa06_radiance', 'Oa07_radiance', 'Oa08_radiance', 'Oa09_radiance', 'Oa10_radiance', 'Oa11_radiance', 'Oa12_radiance', 'Oa13_radiance', 'Oa14_radiance', 'Oa15_radiance', 'Oa16_radiance', 'Oa17_radiance', 'Oa18_radiance', 'Oa19_radiance', 'Oa20_radiance', 'Oa21_radiance', 'quality_flags']


# PROBA-V

In [167]:
pv = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M')

pvImage = ee.Image(pv.filterBounds(sfoPoint).filterDate('2019-05-01', '2019-10-01').sort('CLOUD_COVER_PERCENTAGE').first())
band_names = pvImage.bandNames()
print(band_names.getInfo())

pvBands = ['RED', 'NIR', 'BLUE']
Map.addLayer(pvImage, {'bands': pvBands, 'min': 0.0, 'max': 2000.0}, name = 'PROBA-V Image')

['RED', 'NIR', 'BLUE', 'SWIR', 'NDVI', 'SZA', 'SAA', 'SWIRVAA', 'SWIRVZA', 'VNIRVAA', 'VNIRVZA', 'SM', 'TIME']


# VIIRS

In [168]:
vi = ee.ImageCollection("NOAA/VIIRS/001/VNP09GA")

viImage = ee.Image(vi.filterDate('2019-07-06').first())
viImage = viImage.multiply(0.0001)

band_names = viImage.bandNames()
print(band_names.getInfo())

viBands = ['I1', 'I2', 'I3']
Map.addLayer(viImage, {'bands': viBands, 'min': 0.0, 'max': 1.0}, name = 'VIIRS Image')

['I1', 'I2', 'I3', 'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8', 'M10', 'M11', 'SensorAzimuth', 'SensorZenith', 'SolarAzimuth', 'SolarZenith', 'iobs_res', 'num_observations_1km', 'num_observations_500m', 'obscov_1km', 'obscov_500m', 'orbit_pnt', 'QF1', 'QF2', 'QF3', 'QF4', 'QF5', 'QF6', 'QF7']


# HYPERSPECTRAL SENSOR EO-1

In [169]:
hyp = ee.ImageCollection("EO1/HYPERION") 

hypImages = hyp.filterBounds(sfoPoint).filterDate('2001', '2017')
hypImage = hypImages.mosaic()
hypImage = hypImage.multiply(0.0001)

band_names = hypImage.bandNames()
print(band_names.getInfo())

hypBands = ['B008', 'B009', 'B010']
Map.addLayer(hypImage, {'bands': hypBands, 'min': 0.0, 'max': 1.0}, name = 'Hyperion Image')

['B008', 'B009', 'B010', 'B011', 'B012', 'B013', 'B014', 'B015', 'B016', 'B017', 'B018', 'B019', 'B020', 'B021', 'B022', 'B023', 'B024', 'B025', 'B026', 'B027', 'B028', 'B029', 'B030', 'B031', 'B032', 'B033', 'B034', 'B035', 'B036', 'B037', 'B038', 'B039', 'B040', 'B041', 'B042', 'B043', 'B044', 'B045', 'B046', 'B047', 'B048', 'B049', 'B050', 'B051', 'B052', 'B053', 'B054', 'B055', 'B056', 'B057', 'B077', 'B078', 'B079', 'B080', 'B081', 'B082', 'B083', 'B084', 'B085', 'B086', 'B087', 'B088', 'B089', 'B090', 'B091', 'B092', 'B093', 'B094', 'B095', 'B096', 'B097', 'B098', 'B099', 'B100', 'B101', 'B102', 'B103', 'B104', 'B105', 'B106', 'B107', 'B108', 'B109', 'B110', 'B111', 'B112', 'B113', 'B114', 'B115', 'B116', 'B117', 'B118', 'B119', 'B120', 'B121', 'B122', 'B123', 'B124', 'B125', 'B126', 'B127', 'B128', 'B129', 'B130', 'B131', 'B132', 'B133', 'B134', 'B135', 'B136', 'B137', 'B138', 'B139', 'B140', 'B141', 'B142', 'B143', 'B144', 'B145', 'B146', 'B147', 'B148', 'B149', 'B150', 'B151',

# GPWv411 Basic Demographic Characteristics

In [170]:
dem = ee.ImageCollection("CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics").first()

raster = dem.select('basic_demographic_characteristics');

raster_vis = {
    "max": 1000.0,
    "palette": [
    "ffffe7",
    "86a192",
    "509791",
    "307296",
    "2c4484",
    "000066"
    ],
    "min": 0.0
};

Map.addLayer(raster, raster_vis, name = 'Basic Demographic Characteristics')

# GPWv411 Population Density

In [171]:
pop = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Density").first()

raster = pop.select('population_density');

raster_vis = {
    "max": 1000.0,
    "palette": [
    "ffffe7",
    "FFc869",
    "ffac1d",
    "e17735",
    "f2552c",
    "9f0c21"
    ],
    "min": 200.0
};

Map.addLayer(raster, raster_vis, name = 'Population Density')

# TerraClimate

In [172]:
terra = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')

terraImage = ee.Image(terra
    .filterDate('2019-01-01', '2019-12-31')
    .first());

maximumTemperature = terraImage.select('tmmx');
maximumTemperatureVis = {
    "min": -300.0,
    "max": 300.0,
    "palette": [
        '1a3678', '2955bc', '5699ff', '8dbae9', 'acd1ff', 'caebff', 'e5f9ff',
        'fdffb4', 'ffe6a2', 'ffc969', 'ffa12d', 'ff7c1f', 'ca531a', 'ff0000',
        'ab0000'
    ],
};

Map.addLayer(maximumTemperature, maximumTemperatureVis, name = 'Maximum Temperature');

# PDSI: Palmer Drought Severity Index

In [173]:
dataset = ee.ImageCollection('IDAHO_EPSCOR/PDSI')

datasetImage = ee.Image(dataset
    .filterDate('2019-01-01', '2019-12-31')
    .first());

pdsi = datasetImage.select('pdsi');

pdsiVis = {
    "min": -5.0,
    "max": 10.0,
    "palette": [
        'red', 
        'yellow', 
        'green', 
        'cyan', 
        'blue'
    ],
};

Map.addLayer(pdsi, pdsiVis, name = 'PDSI');

# HUC02: USGS Watershed Boundary Dataset

In [174]:
huc = ee.FeatureCollection('USGS/WBD/2017/HUC02').filterBounds(sfoPoint);

styleParams = {
    "fillColor": '000070',
    "color": '0000be',
    "width": 3.0,
};

#regions = dataset.style(styleParams);

Map.addLayer(huc, styleParams, name = 'USGS Watershed Boundaries');